In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

One of the adavantages of DenseNet is the ability to have a deep narrow network without loss in performance.

In [ ]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('weight>0').reset_index(drop=True)

train.fillna(train.median(),inplace=True)

train['feature_stock_id_sum'] = train['feature_41'] + train['feature_42'] + train['feature_43']
train['feature_1_2_cross'] = train['feature_1']/(train['feature_2']+1e-5)

NUM_TRAIN_EXAMPLES = len(train)

In [ ]:
features = [c for c in train.columns if 'feature' in c]
f_mean = np.nanmedian(train[features[1:]].values,axis=0)

In [ ]:
TRAINING = False
PATH = '../input/highwayjs'

In [ ]:
resp_cols = [c for c in train.columns if 'resp' in c]

In [ ]:
import tensorflow_addons as tfa
import tensorflow as tf
import tensorflow.keras.backend as K

def mish(x):
    return tf.keras.layers.Lambda(lambda x: x*K.tanh(K.softplus(x)))(x)

tf.keras.utils.get_custom_objects().update({'mish': tf.keras.layers.Activation(mish)})

def create_model(input_shape):
    
    inp = tf.keras.layers.Input(input_shape)
    tmp = tf.keras.layers.BatchNormalization()(inp)
    xs = [tmp]
    for _ in range(10):
        if len(xs) > 1:
            tmp = tf.keras.layers.Concatenate(axis=-1)(xs)
        else:
            tmp = xs[0]
        tmp = tf.keras.layers.Dense(32,activation='mish')(tmp)
        tmp = tf.keras.layers.BatchNormalization()(tmp)
        tmp = tf.keras.layers.Dropout(0.2)(tmp)
        xs.append(tmp)
    
    output = tf.keras.layers.Dense(len(resp_cols),activation='sigmoid')(tf.keras.layers.Concatenate()(xs))
    model = tf.keras.models.Model(inp,output)
    optimizer = tfa.optimizers.RectifiedAdam(1e-3)
    model.compile(optimizer, loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001),
                    metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')])
    return model

In [ ]:
import random
import os
def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
X_tr = train.query('date<440')[features].values
y_tr = (train.query('date<440')[resp_cols].values > 0).astype(int)
    
X_val = train.query('date>460')[features].values
y_val = (train.query('date>460')[resp_cols].values > 0).astype(int)
    
if TRAINING:
    metric = {}
    
    for seed in [6, 28, 496, 8128]:
        set_all_seeds(seed)

        model = create_model(X_tr.shape[-1])
        hist = model.fit(X_tr,y_tr,
                         validation_data=(X_val,y_val),
                         epochs=200,
                         batch_size=8192,
                         callbacks=[tf.keras.callbacks.EarlyStopping('val_binary_accuracy',mode='max',patience=20)])
        
        model.save_weights(f'model_{seed}.tf')
        metric[seed] = max(hist.history['binary_accuracy'])
    print(metric)       
else:
    models = []
    for seed in [6, 28, 496, 8128]:
        model = create_model(X_tr.shape[-1])
        model.load_weights(f'{PATH}/model_{seed}.tf')
        model.call = tf.function(model.call, experimental_relax_shapes=True)
        models.append(model)


In [ ]:
blend = np.asarray([0.55722,0.55301,0.56278,0.55624])
blend = blend/sum(blend)

In [ ]:
from tqdm import tqdm
if not TRAINING:
    f = np.median
    import janestreet
    janestreet.competition.make_env.__called__ = False
    env = janestreet.make_env()
    th = 0.503
    for (test_df, pred_df) in tqdm(env.iter_test()):
        if test_df['weight'].item() > 0:
            test_df['feature_stock_id_sum'] = test_df['feature_41'] + test_df['feature_42'] + test_df['feature_43']
            test_df['feature_1_2_cross'] = test_df['feature_1']/(test_df['feature_2']+1e-5)
            x_tt = test_df.loc[:, features].values
            if np.isnan(x_tt[:, 1:].sum()):
                x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
            
            p = sum(blend*np.asarray([f(model(x_tt,training=False).numpy()) for model in models]))
            pred_df.action = np.where(p > th, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)